In [24]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [26]:
url_unique ='https://www.superga.com'
url = "https://www.superga.com/it/collections/stivaletti-donna?filter.p.m.custom.seasons=Primavera%2FEstate"
header = {"User-Agent":"Chrome/39.0"}
print(url)
page = requests.get(url, headers=header)
print(page)

https://www.superga.com/it/collections/stivaletti-donna?filter.p.m.custom.seasons=Primavera%2FEstate
<Response [200]>


In [28]:
soup = BeautifulSoup(page.text)
stivnomi = soup.find_all('a', class_="grid-product__link")
prod_stivali = []

In [48]:
# Configura il browser in modalità headless
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Esegui in modalità headless (senza UI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Lista per salvare i dati estratti da ciascun prodotto
prodotti_dati = []

# Itera su tutti i link nella lista `stivnomi`
for stivale in stivnomi:
    try:
        # Ottieni il link del prodotto
        url_stivale = stivale.get('href')  # Supponendo che `stivnomi` contenga elementi HTML
        url1 = url_unique + url_stivale    # Costruisci l'URL completo
        
        # Carica la pagina
        driver.get(url1)

        # Aspetta che JavaScript carichi la pagina (puoi aggiustare il tempo)
        time.sleep(5)

        # Estrai il contenuto della pagina
        html_content = driver.page_source

        # Analizza la pagina con BeautifulSoup
        soup_stivale = BeautifulSoup(html_content, 'html.parser')

        # Estrai i dati necessari
        name_stiv = soup_stivale.find('h1', class_='h2 product-single__title')
        colore_stiv = soup_stivale.find('span', class_='cb-italic cb-block cb-font-500')
        prezzo_scontato_stiv = soup_stivale.find('span', class_='related__price mr')
        prezzo_originale_stiv = soup_stivale.find('span', class_='related__on-sale')
        colors_divs = soup_stivale.find_all('div', class_='related__color-name skeleton-effect')
        sizes_list = soup_stivale.find_all('ul', class_='related__select-optgroup-options')

        # Gestione valori (evita errori in caso di dati mancanti)
        nomi = name_stiv.text.strip() if name_stiv else "N/A"
        colore = colore_stiv.text.strip() if colore_stiv else "N/A"
        prezzo_scontato = prezzo_scontato_stiv.text.replace("€","").strip() if prezzo_scontato_stiv else "N/A"
        prezzo_originale = prezzo_originale_stiv.text.replace("€","").strip() if prezzo_originale_stiv else "N/A"
        
        # Rimuovi il colore dal nome per ottenere solo il nome del prodotto
        if colore in nomi:
            nomi = nomi.replace(colore, "").strip()

        # Estrai tutti i colori disponibili
        all_colors = [color_div.text.strip() for color_div in colors_divs if color_div.text.strip()]
        
        #  Estrai tutte le taglie disponibili
        sizes_list = soup_stivale.find_all('ul', class_='related__select-optgroup-options')  # Aggiorna la classe se necessario

        # Debug: verifica se sizes_list contiene elementi
        if not sizes_list:
            print(f"Taglie non trovate per il prodotto {url1}")
        
        # Estrai il testo da ogni elemento e filtra solo i numeri
        import re  # Libreria per espressioni regolari
        all_sizes = []
        for ul in sizes_list:
            sizes = ul.find_all('li')  # Supponendo che ogni taglia sia un elemento <li>
            for size in sizes:
                size_text = size.text.strip()
                # Filtra solo numeri usando una regex
                numbers = re.findall(r'\b\d+\b', size_text)  # Cerca numeri interi (come 35, 36)
                all_sizes.extend(numbers)
        
        # Rimuovi duplicati e ordina (opzionale)
        all_sizes = sorted(set(all_sizes), key=int)
        
        # Unisci le taglie in una stringa separata da virgole
        taglie_disponibili = ", ".join(all_sizes) if all_sizes else "N/A"
        
        # Salva i dati per questo prodotto
        prodotti_dati.append({
            'nome': nomi,
            'colore': colore,
            'prezzo_scontato': prezzo_scontato,
            'prezzo_originale': prezzo_originale,
            'colori_disponibili': ", ".join(all_colors),
            'taglie_disponibili': taglie_disponibili,
            'categoria' : "stivaletti",
            'gender':'F',
            'url': url1
        })

    except Exception as e:
        print(f"Errore nell'elaborare il prodotto {url1}: {e}")

# Chiudi il browser
driver.quit()

# Crea un DataFrame con i dati estratti
df_prodotti = pd.DataFrame(prodotti_dati)


# Visualizza il DataFrame
df_prodotti


,nome,colore,prezzo_scontato,prezzo_originale,colori_disponibili,taglie_disponibili,categoria,gender,url
0,3051 ALPINA APEX HIGH,TOTAL BLACK,105.00,N/A,"TOTAL BEIGE NATURAL, TOTAL BLACK","35, 36, 37, 38, 39, 40, 41",stivaletti,F,https://www.superga.com/it/collections/stivale...
1,3051 ALPINA APEX HIGH,TOTAL BEIGE NATURAL,105.00,N/A,"TOTAL BEIGE NATURAL, TOTAL BLACK","35, 36, 37, 38, 39, 40, 41, 42",stivaletti,F,https://www.superga.com/it/collections/stivale...
2,2341 ALPINA NAPPA,FULL BLACK,109.00,N/A,"BEIGE LT EGGSHELL-F AVORIO, BEIGE NATURAL TOTA...","35, 36, 37, 38, 39, 40, 41, 42",stivaletti,F,https://www.superga.com/it/collections/stivale...
3,2341 ALPINA NAPPA,WHITE,109.00,N/A,"BEIGE LT EGGSHELL-F AVORIO, BEIGE NATURAL TOTA...",2,stivaletti,F,https://www.superga.com/it/collections/stivale...
4,2341 ALPINA NAPPA,BLACK-F AVORIO,109.00,N/A,"BEIGE LT EGGSHELL-F AVORIO, BEIGE NATURAL TOTA...","5, 35, 36, 37, 38, 39, 40, 41",stivaletti,F,https://www.superga.com/it/collections/stivale...
5,2341 ALPINA NAPPA,BEIGE NATURAL TOTAL-MATTE-F AVORIO,109.00,N/A,"BEIGE LT EGGSHELL-F AVORIO, BEIGE NATURAL TOTA...","35, 36, 37, 38, 39, 40, 41",stivaletti,F,https://www.superga.com/it/collections/stivale...
6,2641 ALPINA HIGH NAPPA,TOTAL BLACK,139.00,N/A,TOTAL BLACK,"35, 36, 37, 38, 39, 40, 41",stivaletti,F,https://www.superga.com/it/collections/stivale...
7,2341 ALPINA,WHITE,89.00,N/A,"BEIGE LT EGGSHELL-SILVER-F AVORIO, BEIGE LT-F ...","1, 3, 4, 35, 36, 37, 38, 39, 40, 41, 42, 43",stivaletti,F,https://www.superga.com/it/collections/stivale...
8,2341 ALPINA,FULL BLACK,89.00,N/A,"BEIGE LT EGGSHELL-SILVER-F AVORIO, BEIGE LT-F ...","1, 3, 35, 36, 37, 38, 39, 40, 41, 42, 43",stivaletti,F,https://www.superga.com/it/collections/stivale...
9,2341 ALPINA,BLACK-F AVORIO,89.00,N/A,"BEIGE LT EGGSHELL-SILVER-F AVORIO, BEIGE LT-F ...","3, 4, 36, 37, 38, 39, 40",stivaletti,F,https://www.superga.com/it/collections/stivale...


In [50]:
# Salva i dati in un file CSV (opzionale)
df_prodotti.to_csv('Sup_Stiv_Stef.csv', index=False)